# ZEvent 2025 - Data Preparation

Ce notebook combine l'extraction et le nettoyage des données ZEvent 2025.

## Sources de données
- **API ZEvent** : Statistiques officielles, goals, événements
- **SullyGnome** : Métriques Twitch complémentaires

## Outputs
- Données consolidées dans `data/clean/`
- Datasets prêts pour analyse

## 1. Setup et Configuration

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import glob
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configuration des chemins
project_root = Path.cwd().parent  # Remonter depuis notebooks/
data_dir = project_root / 'data'
raw_dir = data_dir / 'raw'
clean_dir = data_dir / 'clean'

print(f"📁 Projet : {project_root}")
print(f"📊 Données raw : {raw_dir}")
print(f"✨ Données clean : {clean_dir}")

📁 Projet : c:\Users\Linwë\Documents\ZEvent2025_data_analyse
📊 Données raw : c:\Users\Linwë\Documents\ZEvent2025_data_analyse\data\raw
✨ Données clean : c:\Users\Linwë\Documents\ZEvent2025_data_analyse\data\clean


## 2. Extraction API ZEvent

Récupération des données depuis l'API officielle ZEvent.

In [7]:
# API ZEvent - URLs et mapping des fichiers
BASE_URL = "https://zevent-api.gdoc.fr"
endpoints = {
    'statistics': f"{BASE_URL}/statistics",
    'donation_goals': f"{BASE_URL}/donation_goals", 
    'event_schedule': f"{BASE_URL}/event_schedule"
}

# Mapping pour les noms de fichiers alternatifs
file_mapping = {
    'statistics': ['statistics.json', 'all_statistics.json', 'individual_streamer_stats.json'],
    'donation_goals': ['donation_goals.json'],
    'event_schedule': ['event_schedule.json']
}

def fetch_zevent_data(endpoint_name, url):
    """Récupère les données d'un endpoint ZEvent ou charge depuis cache"""
    # Vérifier les fichiers possibles pour cet endpoint
    possible_files = file_mapping.get(endpoint_name, [f"{endpoint_name}.json"])
    
    for filename in possible_files:
        raw_file = raw_dir / 'zevent_api' / filename
        if raw_file.exists():
            try:
                with open(raw_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                print(f"📁 {endpoint_name}: Données chargées depuis {filename} ({len(data)} éléments)")
                return data
            except Exception as e:
                print(f"⚠️ Erreur lecture {filename}: {e}")
                continue
    
    # Sinon, essayer de récupérer depuis l'API
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        # Sauvegarde raw
        raw_file = raw_dir / 'zevent_api' / f"{endpoint_name}.json"
        raw_file.parent.mkdir(parents=True, exist_ok=True)
        
        with open(raw_file, 'w', encoding='utf-8') as f:
            json.dump(response.json(), f, indent=2, ensure_ascii=False)
        
        print(f"✅ {endpoint_name}: {len(response.json())} éléments (nouveau)")
        return response.json()
    
    except Exception as e:
        print(f"❌ API {endpoint_name} indisponible: {e}")
        print(f"   → Aucune donnée trouvée pour {endpoint_name}")
        return None

# Extraction des données
zevent_data = {}
for name, url in endpoints.items():
    zevent_data[name] = fetch_zevent_data(name, url)

📁 statistics: Données chargées depuis all_statistics.json (5 éléments)
📁 donation_goals: Données chargées depuis donation_goals.json (5442 éléments)
📁 event_schedule: Données chargées depuis event_schedule.json (57 éléments)


In [5]:
# Diagnostic des données existantes
zevent_api_dir = raw_dir / 'zevent_api'
print(f"📁 Vérification dossier API : {zevent_api_dir}")

if zevent_api_dir.exists():
    existing_files = list(zevent_api_dir.glob('*.json'))
    print(f"📄 Fichiers JSON trouvés: {len(existing_files)}")
    for file in existing_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            print(f"   ✅ {file.name}: {len(data)} éléments")
        except Exception as e:
            print(f"   ❌ {file.name}: Erreur lecture - {e}")
else:
    print("⚠️ Dossier zevent_api inexistant")
    zevent_api_dir.mkdir(parents=True, exist_ok=True)
    print("✅ Dossier créé")

📁 Vérification dossier API : c:\Users\Linwë\Documents\ZEvent2025_data_analyse\data\raw\zevent_api
📄 Fichiers JSON trouvés: 4
   ✅ all_statistics.json: 5 éléments
   ✅ donation_goals.json: 5442 éléments
   ✅ event_schedule.json: 57 éléments
   ✅ individual_streamer_stats.json: 319 éléments


## 3. Traitement SullyGnome

Consolidation des fichiers CSV SullyGnome.

In [9]:
# Chargement et consolidation SullyGnome
sullygnome_dir = raw_dir / 'sullygnome'
csv_files = list(sullygnome_dir.glob('*.csv'))

print(f"📁 Fichiers SullyGnome trouvés: {len(csv_files)}")

# Consolidation
all_pages = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    all_pages.append(df)
    print(f"   {csv_file.name}: {len(df)} streamers")

if all_pages:
    df_sullygnome = pd.concat(all_pages, ignore_index=True)
    print(f"\n📊 Total SullyGnome: {len(df_sullygnome)} streamers")
    
    # Nettoyage basique
    df_sullygnome = df_sullygnome.drop_duplicates(subset=['Channel'])
    df_sullygnome['Channel'] = df_sullygnome['Channel'].str.lower().str.strip()
    
    print(f"✨ Après dédoublonnage: {len(df_sullygnome)} streamers uniques")
else:
    print("⚠️ Aucun fichier SullyGnome trouvé")
    df_sullygnome = pd.DataFrame()

📁 Fichiers SullyGnome trouvés: 5
   ZEVENT - most watched Twitch channels - SullyGnome - page 1.csv: 100 streamers
   ZEVENT - most watched Twitch channels - SullyGnome - page 2.csv: 100 streamers
   ZEVENT - most watched Twitch channels - SullyGnome - page 3.csv: 100 streamers
   ZEVENT - most watched Twitch channels - SullyGnome - page 4.csv: 100 streamers
   ZEVENT - most watched Twitch channels - SullyGnome - page 5.csv: 63 streamers

📊 Total SullyGnome: 463 streamers
✨ Après dédoublonnage: 463 streamers uniques


## 4. Nettoyage et Enrichissement

Préparation des datasets finaux.

In [16]:
# Traitement des données streamers ZEvent
streamers_dir = clean_dir / 'streamers'
streamers_dir.mkdir(exist_ok=True)

# Chargement et analyse du fichier individual_streamer_stats
try:
    with open(raw_dir / 'zevent_api' / 'individual_streamer_stats.json', 'r', encoding='utf-8') as f:
        streamer_stats = json.load(f)
    
    print(f"📊 Type de données: {type(streamer_stats)}")
    if isinstance(streamer_stats, dict):
        print(f"📊 Clés disponibles: {list(streamer_stats.keys())}")
        # Si c'est un dict, prendre les valeurs ou une clé spécifique
        if 'streamers' in streamer_stats:
            streamer_list = streamer_stats['streamers']
        elif isinstance(list(streamer_stats.values())[0], list):
            streamer_list = list(streamer_stats.values())[0]
        else:
            streamer_list = list(streamer_stats.values())
    elif isinstance(streamer_stats, list):
        streamer_list = streamer_stats
    else:
        streamer_list = []
    
    if streamer_list and len(streamer_list) > 0:
        df_zevent = pd.DataFrame(streamer_list)
        print(f"📊 Streamers ZEvent: {len(df_zevent)} streamers")
        print(f"📋 Colonnes: {list(df_zevent.columns)}")
        
        # Nettoyage username pour matching
        username_col = None
        for col in ['username', 'name', 'channel', 'streamer']:
            if col in df_zevent.columns:
                username_col = col
                break
        
        if username_col:
            df_zevent['username_clean'] = df_zevent[username_col].str.lower().str.strip()
            print(f"✅ Colonne username_clean créée depuis {username_col}")
        
        # Sauvegarde streamers base
        df_zevent.to_csv(streamers_dir / 'streamers_data.csv', index=False)
        print(f"💾 Sauvegarde: streamers_data.csv")
        
        # Enrichissement avec SullyGnome si disponible
        if not df_sullygnome.empty and 'username_clean' in df_zevent.columns:
            # Matching par username
            df_enriched = df_zevent.merge(
                df_sullygnome.rename(columns={'Channel': 'username_clean'}),
                on='username_clean',
                how='left'
            )
            
            # Sauvegarde enrichie
            df_enriched.to_csv(streamers_dir / 'zevent_sullygnome_enriched.csv', index=False)
            print(f"💾 Sauvegarde: zevent_sullygnome_enriched.csv")
            
            # Stats de matching
            sullygnome_cols = [col for col in df_enriched.columns if col in df_sullygnome.columns]
            if sullygnome_cols:
                matched = df_enriched.dropna(subset=[sullygnome_cols[0]]).shape[0]
                print(f"🔗 Matching réussi: {matched}/{len(df_zevent)} streamers")
        
        # Sauvegarde SullyGnome consolidé
        if not df_sullygnome.empty:
            df_sullygnome.to_csv(streamers_dir / 'sullygnome_consolidated.csv', index=False)
            print(f"💾 Sauvegarde: sullygnome_consolidated.csv")
            
    else:
        print("❌ Aucune donnée streamer trouvée")
        
except Exception as e:
    print(f"❌ Erreur traitement streamers: {e}")
    # Utiliser les données existantes si le fichier CSV existe déjà
    existing_streamers = streamers_dir / 'streamers_data.csv'
    if existing_streamers.exists():
        print("   → Utilisation des données existantes")
    else:
        print("   → Aucune donnée disponible")

📊 Type de données: <class 'dict'>
📊 Clés disponibles: ['0uahleouff', '2old4stream', 'adyboo', 'airkagaming', 'alexclick', 'allmight__one', 'alphacast', 'anaee', 'anaonair', 'angledroit', 'antistar', 'antoinedaniel', 'arnaquemoisitupeux', 'artemize', 'as2pik', 'avamind', 'azghaaar', 'bagherajones', 'barbe___douce', 'bartgeo', 'bastiui', 'bclv4', 'behop_veek', 'benzaie', 'bigbadhater_fr', 'bluestardust_', 'boitameu', 'boomnasty_', 'bouddhiste', 'brickmitri', 'broocoline', 'brunouniverssimu', 'brybry', 'byilhann', 'cailloute', 'captainfracas', 'chaab', 'chaton_sauvage_', 'chester_ttv', 'chezbubulle', 'chowh1', 'christellelebaillyauteur', 'citronviolet', 'clara_l_exploratrice', 'clem_mlrt', 'clemovitch', 'clubpingouin_trash', 'coffee', 'crocodyletv', 'crosswax', 'cruelladk', 'damdamlive', 'dams_ow', 'david_kyden', 'deavild', 'deejaymakina', 'did0us', 'doigby', 'domingo', 'doxtv75', 'drakeoz_', 'drfeelgood', 'drfrenesy', 'echyr_', 'elbe_poly', 'elfyaa', 'elizyatv', 'elnoraeleo', 'elyhonair'

## 5. Sauvegarde Données Événements

In [17]:
# Sauvegarde goals et événements
events_dir = clean_dir / 'events'
events_dir.mkdir(exist_ok=True)

# Donation goals
if zevent_data.get('donation_goals'):
    df_goals = pd.DataFrame(zevent_data['donation_goals'])
    df_goals.to_csv(events_dir / 'donation_goals.csv', index=False)
    print(f"💾 Goals sauvegardés: {len(df_goals)} objectifs")

# Event schedule  
if zevent_data.get('event_schedule'):
    df_schedule = pd.DataFrame(zevent_data['event_schedule'])
    df_schedule.to_csv(events_dir / 'event_schedule.csv', index=False)
    print(f"💾 Planning sauvegardé: {len(df_schedule)} événements")

💾 Goals sauvegardés: 5442 objectifs
💾 Planning sauvegardé: 57 événements


## 6. Résumé des Données Préparées

In [18]:
# Résumé final
print("\n" + "="*50)
print("📊 RÉSUMÉ DES DONNÉES PRÉPARÉES")
print("="*50)

# Vérification des fichiers créés
clean_files = list(clean_dir.rglob('*.csv'))
for file in clean_files:
    df_check = pd.read_csv(file)
    relative_path = file.relative_to(clean_dir)
    print(f"✅ {relative_path}: {len(df_check)} lignes, {len(df_check.columns)} colonnes")

print(f"\n📁 Tous les fichiers dans: {clean_dir}")
print("✨ Données prêtes pour l'analyse !")


📊 RÉSUMÉ DES DONNÉES PRÉPARÉES
✅ events\donation_goals.csv: 5442 lignes, 10 colonnes
✅ events\event_schedule.csv: 57 lignes, 7 colonnes
✅ streamers\streamers_data.csv: 319 lignes, 5 colonnes
✅ streamers\streamers_detailed_stats.csv: 326 lignes, 15 colonnes
✅ streamers\sullygnome_consolidated.csv: 463 lignes, 12 colonnes
✅ streamers\sullygnome_raw_consolidated.csv: 463 lignes, 15 colonnes
✅ streamers\sullygnome_zevent_consolidated.csv: 463 lignes, 19 colonnes
✅ streamers\zevent_sullygnome_enriched.csv: 316 lignes, 26 colonnes
✅ temporal\donations_evolution.csv: 110 lignes, 3 colonnes
✅ temporal\viewers_evolution.csv: 111 lignes, 3 colonnes

📁 Tous les fichiers dans: c:\Users\Linwë\Documents\ZEvent2025_data_analyse\data\clean
✨ Données prêtes pour l'analyse !
